# Projet

## Apercu du projet

Vous avez rejoint une nouvelle équipe dans le secteur de la banque de détail, qui connaît actuellement des taux de défaut plus élevés que prévu sur les prêts personnels. Les prêts personnels sont une source de revenus importante pour les banques, mais ils comportent le risque inhérent que les emprunteurs puissent faire défaut. Un défaut de paiement se produit lorsqu'un emprunteur cesse de faire les paiements requis sur une dette.

## Objectif : 

L'équipe de risque analyse le portefeuille de prêts existants pour prévoir les défauts potentiels futurs et estimer la perte attendue. L'objectif principal est de construire un modèle prédictif qui estime la probabilité de défaut pour chaque client en fonction de ses caractéristiques. Des prédictions précises permettront à la banque d'allouer suffisamment de capital pour couvrir les pertes potentielles, maintenant ainsi la stabilité financière.

### 1. Exploration du Dataset

In [87]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mlflow 

In [88]:
# Chargement du fichier CSV
fichier = "Loan_Data.csv"
df = pd.read_csv(fichier)

In [68]:
# Aperçu général
print("\n Aperçu des données :")
df.head()


 Aperçu des données :


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [6]:
print("\n Dimensions :", df.shape)
print("\n Types de données :")
print(df.dtypes)


 Dimensions : (10000, 8)

 Types de données :
customer_id                   int64
credit_lines_outstanding      int64
loan_amt_outstanding        float64
total_debt_outstanding      float64
income                      float64
years_employed                int64
fico_score                    int64
default                       int64
dtype: object


In [9]:
# Statistiques descriptives (sans 'customer_id')

print("\n Statistiques descriptives :")
df.describe().drop(columns=['customer_id'])


 Statistiques descriptives :


,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.461200,4159.677034,8718.916797,70039.901401,4.552800,637.557700,0.185100
std,1.743846,1421.399078,6627.164762,20072.214143,1.566862,60.657906,0.388398
min,0.000000,46.783973,31.652732,1000.000000,0.000000,408.000000,0.000000
25%,0.000000,3154.235371,4199.836020,56539.867903,3.000000,597.000000,0.000000
50%,1.000000,4052.377228,6732.407217,70085.826330,5.000000,638.000000,0.000000
75%,2.000000,5052.898103,11272.263740,83429.166133,6.000000,679.000000,0.000000
max,5.000000,10750.677810,43688.784100,148412.180500,10.000000,850.000000,1.000000


In [10]:
# Analyse des valeurs manquantes
print("\n Valeurs manquantes :")
print(df.isnull().sum())


 Valeurs manquantes :
customer_id                 0
credit_lines_outstanding    0
loan_amt_outstanding        0
total_debt_outstanding      0
income                      0
years_employed              0
fico_score                  0
default                     0
dtype: int64


In [13]:
# Vérifier s'il y a des doublons dans le DataFrame
nb_doublons = df.duplicated().sum()

if nb_doublons > 0:
    print(f"Il y a {nb_doublons} lignes dupliquées dans le dataset.")
else:
    print(" Aucun doublon détecté dans le dataset.")

 Aucun doublon détecté dans le dataset.


### 2. Pré-traitement

In [89]:
# Définition de la variable cible
target = "default"  # 🔁 adapte ici si ta colonne s'appelle autrement (ex: "loan_status")
X = df.drop(columns=[target, "customer_id"])
y = df[target]

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 X_train: {X_train.shape}, X_test: {X_test.shape}")

# Normalisation (StandardScaler)
# On standardise pour centrer-réduire les features : (x - mean)/std
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Normalisation effectuée (StandardScaler appliqué sur X_train et X_test)")

📊 X_train: (8000, 6), X_test: (2000, 6)
Normalisation effectuée (StandardScaler appliqué sur X_train et X_test)


### 3. Test de 3 modèles de ML avec MLFLOW

In [ ]:
# starts an MLflow server locally.
# mlflow server --host 127.0.0.1 --port 8080

In [70]:
import mlflow 

from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

In [21]:
# In order to connect to the tracking server, we’ll need to use the uri that we assigned the server when we started it.

client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

### Logging our runs with MLflow

#### *ML 1 : Decision Tree Experiment*

In [24]:
experiment_description_dt = (
    "Modèle Decision Tree pour la prédiction de défaut de prêt. "
    "Cet experiment contient les runs liés au modèle Decision Tree."
)

experiment_tags_dt = {
    "project_name": "loan-default-prediction",
    "model_type": "DecisionTree",
    "team": "mlops-bank",
    "project_quarter": "Q4-2025",
    "mlflow.note.content": experiment_description_dt,
}

experiment_ML_decision_tree = client.create_experiment(
    name="DecisionTree_Experiment_ML1",
    tags=experiment_tags_dt,
)

In [77]:
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
)
import numpy as np
import joblib

In [78]:
# This function call sets the global tracking URI for the current session.
# It’s a convenient way to configure the tracking server URI without creating a separate client instance.

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [79]:
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
ml1_experiment = mlflow.set_experiment("DecisionTree_Experiment_ML1")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "DecisionTree_run"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_DecisionTree"

In [80]:
# ======================================================
# 🌳 Entraînement du modèle Decision Tree
# ======================================================
params = {
    "criterion": "gini",
    "max_depth": 5,
    "min_samples_split": 4,
    "min_samples_leaf": 2,
    "random_state": 42,
}

# Démarrage du run MLflow
with mlflow.start_run(run_name=run_name) as run:
    # Initialisation et entraînement du modèle
    dt = DecisionTreeClassifier(**params)
    dt.fit(X_train_scaled, y_train)

    # Prédictions sur le test set
    y_pred = dt.predict(X_test_scaled)
    y_proba = dt.predict_proba(X_test_scaled)[:, 1]

    # Calcul des métriques
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba)

    # On regroupe toutes les métriques dans un dict
    metrics = {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc,
    }

    # ======================================================
    # 📊 Logging MLflow
    # ======================================================
    # Log des hyperparamètres
    mlflow.log_params(params)

    # Log des métriques
    mlflow.log_metrics(metrics)

    # 🧠 Sauvegarde manuelle du modèle au format .pkl
    joblib.dump(dt, "model_ML1.pkl")

    # 📂 Envoi du modèle en tant qu’artifact dans MLflow
    mlflow.log_artifact("model_ML1.pkl")

    # Exemple de jeu de données pour déduire le schéma du modèle
    input_example = X_test_scaled[:5]

    # Définition de la signature du modèle (schéma des inputs/outputs)
    from mlflow.models.signature import infer_signature
    signature = infer_signature(X_test_scaled, dt.predict(X_test_scaled))

    # Logging complet du modèle avec signature + input example
    mlflow.sklearn.log_model(
        sk_model=dt,
        name="rf_DecisionTree",
        input_example=input_example,
        signature=signature
    )

    print("✅ Modèle Decision Tree enregistré dans MLflow !")
    print("📊 Métriques :")
    for k, v in metrics.items():
        print(f"{k:10s} : {v:.3f}")

✅ Modèle Decision Tree enregistré dans MLflow !
📊 Métriques :
accuracy   : 0.997
precision  : 0.992
recall     : 0.989
f1_score   : 0.991
roc_auc    : 0.999
🏃 View run DecisionTree_run at: http://127.0.0.1:8080/#/experiments/704114334643502776/runs/12d8369e6d214c61ade2d5444d83f494
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/704114334643502776


#### *ML 2 : Logistic Regression Experiment*

In [45]:
experiment_description_lr = (
    "Modèle de Régression Logistique pour la prédiction de défaut de prêt. "
    "Cet experiment contient les runs liés au modèle Logistic Regression."
)

experiment_tags_lr = {
    "project_name": "loan-default-prediction",
    "model_type": "LogisticRegression",
    "team": "mlops-bank",
    "project_quarter": "Q4-2025",
    "mlflow.note.content": experiment_description_lr,
}

experiment_logistic_regression = client.create_experiment(
    name="LogisticRegression_Experiment_ml2",
    tags=experiment_tags_lr,
)

In [90]:
# This function call sets the global tracking URI for the current session.
# It’s a convenient way to configure the tracking server URI without creating a separate client instance.

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [91]:
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
ml2_experiment = mlflow.set_experiment("LogisticRegression_Experiment_ml2")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "LogisticRegression_run"

# Define an artifact path that the model will be saved to.
artifact_path = "lr_LogisticRegression"

In [92]:
from sklearn.linear_model import LogisticRegression

# ======================================================
# ⚙️ Paramètres du modèle
# ======================================================
params = {
    "penalty": "l2",
    "solver": "lbfgs",
    "max_iter": 1000,
    "random_state": 42
}

# ======================================================
# 🚀 Entraînement du modèle Logistic Regression
# ======================================================
with mlflow.start_run(run_name=run_name) as run:
    # Initialisation du modèle
    lr = LogisticRegression(**params)
    lr.fit(X_train_scaled, y_train)

    # Prédictions
    y_pred = lr.predict(X_test_scaled)
    y_proba = lr.predict_proba(X_test_scaled)[:, 1]

    # Calcul des métriques
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba)

    metrics = {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc,
    }

    # ======================================================
    # 📊 Logging MLflow
    # ======================================================
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)

    # 🔹 Sauvegarde manuelle du modèle en .pkl
    joblib.dump(lr, "model_ML2.pkl")
    mlflow.log_artifact("model_ML2.pkl")

    # 🔹 Enregistrement du modèle MLflow (avec schéma)
    input_example = X_test_scaled[:5]
    signature = infer_signature(X_test_scaled, lr.predict(X_test_scaled))
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path=artifact_path,
        input_example=input_example,
        signature=signature
    )

    print("✅ Modèle Régression Logistique enregistré dans MLflow !")
    print("📊 Métriques :")
    for k, v in metrics.items():
        print(f"{k:10s} : {v:.3f}")

2025/10/16 11:55:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Modèle Régression Logistique enregistré dans MLflow !
📊 Métriques :
accuracy   : 0.999
precision  : 1.000
recall     : 0.995
f1_score   : 0.997
roc_auc    : 1.000
🏃 View run LogisticRegression_run at: http://127.0.0.1:8080/#/experiments/809115022134385542/runs/8ff205be06f2411395807f9776117cff
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/809115022134385542


#### *ML 3 : Random Forest Experiment*

In [55]:
experiment_description_rf = (
    "Modèle Random Forest pour la prédiction de défaut de prêt. "
    "Cet experiment contient les runs liés au modèle Random Forest."
)

experiment_tags_rf = {
    "project_name": "loan-default-prediction",
    "model_type": "RandomForest",
    "team": "mlops-bank",
    "project_quarter": "Q4-2025",
    "mlflow.note.content": experiment_description_rf,
}

experiment_random_forest = client.create_experiment(
    name="RandomForest_Experiment_ml3",
    tags=experiment_tags_rf,
)

In [84]:
# This function call sets the global tracking URI for the current session.
# It’s a convenient way to configure the tracking server URI without creating a separate client instance.

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [85]:
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
ml3_experiment = mlflow.set_experiment("RandomForest_Experiment_ml3")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "RandomForest_run"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_RandomForest"

In [86]:
from sklearn.ensemble import RandomForestClassifier

# ======================================================
# ⚙️ Paramètres du modèle Random Forest
# ======================================================
params = {
    "n_estimators": 100,
    "max_depth": 8,
    "min_samples_split": 4,
    "min_samples_leaf": 2,
    "bootstrap": True,
    "random_state": 42
}

# ======================================================
# 🚀 Entraînement du modèle Random Forest
# ======================================================
with mlflow.start_run(run_name=run_name) as run:
    # Initialisation du modèle
    rf = RandomForestClassifier(**params)
    rf.fit(X_train_scaled, y_train)

    # Prédictions
    y_pred = rf.predict(X_test_scaled)
    y_proba = rf.predict_proba(X_test_scaled)[:, 1]

    # Calcul des métriques
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba)

    metrics = {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc,
    }

    # ======================================================
    # 📊 Logging MLflow
    # ======================================================
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)

    # 🔹 Sauvegarde manuelle du modèle en .pkl
    joblib.dump(rf, "model_ML3.pkl")
    mlflow.log_artifact("model_ML3.pkl")

    # 🔹 Signature et input_example
    input_example = X_test_scaled[:5]
    signature = infer_signature(X_test_scaled, rf.predict(X_test_scaled))

    # 🔹 Log du modèle complet dans MLflow
    mlflow.sklearn.log_model(
        sk_model=rf,
        artifact_path=artifact_path,
        input_example=input_example,
        signature=signature
    )


    print("✅ Modèle Random Forest enregistré dans MLflow !")
    print("📊 Métriques :")
    for k, v in metrics.items():
        print(f"{k:10s} : {v:.3f}")

2025/10/16 11:49:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Modèle Random Forest enregistré dans MLflow !
📊 Métriques :
accuracy   : 0.996
precision  : 0.992
recall     : 0.984
f1_score   : 0.988
roc_auc    : 1.000
🏃 View run RandomForest_run at: http://127.0.0.1:8080/#/experiments/617615322798623046/runs/f244859c530045e8b2e6ca2d21deb45c
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/617615322798623046
